In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import pickle
import os
folder_pickle= '../../data/pickle'
folder_text= '../../data/text'

In [13]:
data = pd.read_csv('data_fusion.csv')
dataClean= data.dropna()


In [14]:
print(dataClean.shape)
print(dataClean.shape)

(174295, 19)
(174295, 19)


In [15]:

if not os.path.exists(folder_pickle):
    os.makedirs(folder_pickle, exist_ok=True)
if not os.path.exists(folder_text):
    os.makedirs(folder_text, exist_ok=True)


In [16]:
for i in range(1, 6):
    with open(os.path.join(folder_text, f"{i}star.txt"), 'w') as f:
        f.write('')

In [17]:
from tqdm import tqdm

# Parcourir le dataframe et ajouter le commentaire dans le fichier correspondant à la note
for index, row in tqdm(data.iterrows(), total=data.shape[0]):
    note = int(row['revue/score'])
    commentaire = row['revue/texte']
    if pd.notna(commentaire):  # Check if commentaire is not NaN
        with open(os.path.join(folder_text, f"{note}star.txt"), 'a', encoding='utf-8') as f:
            f.write(commentaire + '\n')

100%|██████████| 2999782/2999782 [12:37<00:00, 3962.39it/s]


In [18]:

%pip install nltk

import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Télécharger les ressources nécessaires
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in words]
    return ' '.join(lemmatized_words)

# Exemple d'utilisation
text = "The striped bats are hanging on their feet for best"
lemmatized_text = lemmatize_text(text)
print(lemmatized_text)

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
The strip bat be hang on their foot for best



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\mathi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\mathi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\mathi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mathi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mathi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\U

In [19]:
for i in range(1, 6):
    input_file_path = os.path.join(folder_text, f"{i}star.txt")
    output_file_path = os.path.join(folder_text, f"{i}star_lemmatized.txt")
    
    with open(input_file_path, 'r', encoding='utf-8') as input_file:
        content = input_file.read()
    
    lemmatized_content = lemmatize_text(content)
    
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        output_file.write(lemmatized_content)

In [8]:
# Initialiser le vecteur TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Extraire les textes des fichiers lemmatisés
texts = []
for i in range(1, 6):
    with open(os.path.join(folder_text, f"{i}star_lemmatized.txt"), 'r', encoding='utf-8') as file:
        texts.append(file.read())

# Ajuster et transformer les textes en une matrice TF-IDF
tfidf_matrix = tfidf_vectorizer.fit_transform(texts)

# Renommer les features pour qu'elles correspondent aux fichiers
feature_names = [f"{i}star" for i in range(1, 6)]

# Afficher les termes et la matrice TF-IDF
print("Features names:", feature_names)
print("TF-IDF Matrix:\n", tfidf_matrix.toarray())

# Enregistrer la matrice TF-IDF dans un fichier pickle
with open(os.path.join(folder_pickle, 'tfidf_matrix.pkl'), 'wb') as file:
    pickle.dump(tfidf_matrix, file)
    # Enregistrer le TF-IDF vectorizer dans un fichier pickle
with open(os.path.join(folder_pickle, 'tfidf_vectorizer.pkl'), 'wb') as file:
    pickle.dump(tfidf_vectorizer, file)

Features names: ['1star', '2star', '3star', '4star', '5star']
TF-IDF Matrix:
 [[3.24114647e-04 9.14055282e-04 2.22759208e-06 ... 7.79142017e-07
  7.79142017e-07 0.00000000e+00]
 [1.64630849e-04 6.71443792e-04 4.16786960e-07 ... 0.00000000e+00
  0.00000000e+00 1.74934847e-06]
 [1.14628101e-04 5.72681074e-04 9.18862534e-07 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.94323596e-05 6.12585684e-04 3.11907171e-07 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.26408830e-04 6.37399027e-04 6.32465794e-07 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [9]:
from sklearn.metrics.pairwise import cosine_similarity
#
# Utiliser la matrice TF-IDF pour faire un commentaire de test
test_comments = [
    "This book provides a comprehensive overview of the subject and is very well written.",
    "The storyline was captivating and the characters were well developed.",
    "I found the book to be quite boring and not engaging at all.",
    "An excellent read with lots of useful information.",
    "The plot was predictable and the writing style was not to my liking.",
    "The book was too long and filled with unnecessary details.",
    "I didn't like the main character and found the plot confusing.",
    "The writing was poor and the story didn't hold my interest.",
    "I wouldn't recommend this book to anyone.",
    "The book was a waste of time and money."
]
# Charger la matrice TF-IDF à partir du fichier pickle
with open(os.path.join(folder_pickle, 'tfidf_matrix.pkl'), 'rb') as file:
    tfidf_matrix = pickle.load(file)
# Transformer le commentaire de test en vecteur TF-IDF
with open(os.path.join(folder_pickle, 'tfidf_vectorizer.pkl'), 'rb') as file:
    tfidf_vectorizer_tester = pickle.load(file)

test_vectors = tfidf_vectorizer_tester.transform(test_comments)

# Calculer la similarité cosinus entre les commentaires de test et les textes existants
for idx, test_vector in enumerate(test_vectors):
    similarities = cosine_similarity(test_vector, tfidf_matrix)
    max_similarity_index = similarities.argmax()
    print(f"Commentaire de test {idx + 1}:")
    print(f"Similarité cosinus la plus élevée avec {feature_names[max_similarity_index]}: {similarities[0][max_similarity_index]}")
    print("\n")

Commentaire de test 1:
Similarité cosinus la plus élevée avec 5star: 0.43549307023583755


Commentaire de test 2:
Similarité cosinus la plus élevée avec 4star: 0.44310138199965987


Commentaire de test 3:
Similarité cosinus la plus élevée avec 1star: 0.55323483680274


Commentaire de test 4:
Similarité cosinus la plus élevée avec 5star: 0.173957298511708


Commentaire de test 5:
Similarité cosinus la plus élevée avec 4star: 0.4526659202881725


Commentaire de test 6:
Similarité cosinus la plus élevée avec 5star: 0.3636150330947245


Commentaire de test 7:
Similarité cosinus la plus élevée avec 4star: 0.45732947824172954


Commentaire de test 8:
Similarité cosinus la plus élevée avec 4star: 0.44013352936201416


Commentaire de test 9:
Similarité cosinus la plus élevée avec 1star: 0.2609118878250282


Commentaire de test 10:
Similarité cosinus la plus élevée avec 5star: 0.4938654797819173


